# NLP beginer

https://github.com/proger-n/DS_01_Tweets/blob/main/src/tweets.ipynb

### Introduction
NLP (natural language processing) — это область знаний и ряд методов и алгоритмов, которые помогают обрабатывать текстовые данные и извлекать из них информацию. Вы уже работали со структурированными данными — таблицами, полными непрерывных и категориальных признаков. Текст является примером полуструктурированных данных. С ним нужно что-то делать, чтобы использовать его, например, в задачах машинного обучения.

ЦЕЛЬ РАБОТЫ - предварительная обработка тектовых данных и обучение с помощью различных классификаторов для решения задачи классификации твитов на три класса (негативный, отрицательный и нейтральный)

### 1. Предварительная обработка текста:
Здесь перед подачей в классификатор необходимо предобработать текст, есть следующие способы:
- [Term Frequency-Inverse Document Frequency (TF-IDF)](https://habr.com/ru/companies/otus/articles/755772/) - это один из наиболее распространенных и мощных методов для извлечения признаков из текстовых данных. TF-IDF вычисляет важность каждого слова в документе относительно количества его употреблений в данном документе и во всей коллекции текстов. Этот метод позволяет выделить ключевые слова и понять, какие слова имеют больший вес для определенного документа в контексте всей коллекции.
Перед тем как вычислять TF-IDF, мы должны выполнить предварительную обработку, такую как удаление стоп-слов, приведение к нижнему регистру и токенизация — разбиение текстов на отдельные слова или токены.
- Стемминг (stemming) - это грубый эвристический процесс, который отрезает «лишнее» от корня слов, часто это приводит к потере словообразовательных суффиксов. Например слово "Коты", станет "Кот"
- Лематизация - это более тонкий процесс, который использует словарь и морфологический анализ, чтобы в итоге привести слово к его канонической форме – лемме.

Примеры лематизации и стемминга: Слово good – это лемма для слова better. Стеммер не увидит эту связь, так как здесь нужно сверяться со словарем.
Слово play – это базовая форма слова playing. Тут справятся и стемминг, и лемматизация.
Слово meeting может быть как нормальной формой существительного, так и формой глагола to meet, в зависимости от контекста.
В отличие от стемминга, лемматизация попробует выбрать правильную лемму,опираясь на контекст.

## 0.Import

In [4]:
import pandas as pd
import zipfile

## 1. Подготовка данных
Данные хранятся в архиве. в Архиве три файла csv. В одном файле негативные отзыва, в другом позитивные и в третьем нейтральные. csv файлы содержат одну строку и много столбцов, поэтому нужно будет трансопнировать и привести всё в одну таблицу

In [30]:
# Распаковываем zip архив
with zipfile.ZipFile('../datasets/p00_tweets.zip', 'r') as zip_ref:
    zip_ref.extractall('data')
    
# Читаем файлы в pandas
positive_tweets = pd.read_csv('data/processedPositive.csv')
print(positive_tweets.shape)

neutral_tweets = pd.read_csv('data/processedNeutral.csv')
print(neutral_tweets.shape)

negative_tweets = pd.read_csv('data/processedNegative.csv')
print(negative_tweets.shape)

(0, 1186)
(0, 1570)
(0, 1117)


### 1.1. Приведем все данные в одну таблицу.
Для этого транспонируем исходные таблицы и назначим индексы
- 1 - негативный твит
- 2 - нейтральный твит
- 3 - положительный твит

In [31]:
def prepare_data(df, target_index):
    df_new = df.T.reset_index().rename(columns={'index': 'tweet'})
    df_new['target'] = target_index
    return df_new

In [32]:
negative_tweets = prepare_data(negative_tweets, target_index=1)
neutral_tweets = prepare_data(neutral_tweets, target_index=2)
positive_tweets = prepare_data(positive_tweets, target_index=3)

df = pd.concat([negative_tweets, neutral_tweets, positive_tweets], ignore_index=True)
df.head(200)

,tweet,target
0,How unhappy some dogs like it though,1
1,talking to my over driver about where I'm goin...,1
2,Does anybody know if the Rand's likely to fall...,1
3,I miss going to gigs in Liverpool unhappy,1
4,There isnt a new Riverdale tonight ? unhappy,1
...,...,...
195,Beyond sad unhappy,1
196,Goodnight princess,1
197,I love you have fun with your friends,1
198,YG should have sent them to MCD. I want to see...,1


### 1.2. чистим твиты
убираем все символы кроме букв и преобразуем к нижнему регистру

In [34]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r'[^a-zA-Z ]', '', regex=True)
    df[text_field] = df[text_field].str.lower()
    return df

df = standardize_text(df, 'tweet')

df

,tweet,target
0,how unhappy some dogs like it though,1
1,talking to my over driver about where im going...,1
2,does anybody know if the rands likely to fall ...,1
3,i miss going to gigs in liverpool unhappy,1
4,there isnt a new riverdale tonight unhappy,1
...,...,...
3868,thanks for the recent follow happy to connect ...,3
3869,top engaged members this week happy,3
3870,ngam to weeks left for cadet pilot exam cryin...,3
3871,great youre welcome josh happy adam,3
